In [1]:
import pandas as pd
import numpy as np
from dijkstar import Graph, find_path
from itertools import chain
from tqdm import tqdm
import re

In [2]:
with open('numkeypad.txt') as file:
    nkp = file.read().split('\n')
    nkp = pd.DataFrame([x for x in [[x for x in list(line) if x not in ['+', '-', '|', ' ']] for line in nkp] if x != []])
    nkp.iloc[3,:] = [np.nan,'0', 'A']
    file.close()    
nkp                             

,0,1,2
0,7,8,9
1,4,5,6
2,1,2,3
3,NaN,0,A


In [3]:
with open('dirkeypad.txt') as file:
    dkp = file.read().split('\n')
    dkp = pd.DataFrame([x for x in [[x for x in list(line) if x not in ['+', '-', '|', ' ']] for line in dkp] if x != []])
    dkp.iloc[0,:] = [np.nan,'^', 'A']
    file.close()    
dkp    

,0,1,2
0,NaN,^,A
1,<,v,>


In [4]:
with open('Sample.txt') as file:
    input = file.read().split('\n')
    file.close()
input

['029A', '980A', '179A', '456A', '379A']

In [5]:
pl = list(chain.from_iterable([[(x, y) for x in range(len(nkp))] for y in range(len(nkp.columns))]))
pl = [x for x in pl if nkp.iloc[x[0], x[1]] == nkp.iloc[x[0], x[1]]]
graph_nkp = Graph()

for p in tqdm(pl):
    iterator = iter([0,1,0,-1,1,0,-1,0])
    i = 0
    while i < 4:
        i += 1
        n = (p[0]+next(iterator), p[1] + next(iterator))
        if n in pl:    
            graph_nkp.add_edge(p, n, 1)
graph_nkp

100%|██████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<?, ?it/s]


{(0, 0): {(0, 1): 1, (1, 0): 1}, (1, 0): {(1, 1): 1, (2, 0): 1, (0, 0): 1}, (2, 0): {(2, 1): 1, (1, 0): 1}, (0, 1): {(0, 2): 1, (0, 0): 1, (1, 1): 1}, (1, 1): {(1, 2): 1, (1, 0): 1, (2, 1): 1, (0, 1): 1}, (2, 1): {(2, 2): 1, (2, 0): 1, (3, 1): 1, (1, 1): 1}, (3, 1): {(3, 2): 1, (2, 1): 1}, (0, 2): {(0, 1): 1, (1, 2): 1}, (1, 2): {(1, 1): 1, (2, 2): 1, (0, 2): 1}, (2, 2): {(2, 1): 1, (3, 2): 1, (1, 2): 1}, (3, 2): {(3, 1): 1, (2, 2): 1}}

In [6]:
pl = list(chain.from_iterable([[(x, y) for x in range(len(dkp))] for y in range(len(dkp.columns))]))
pl = [x for x in pl if dkp.iloc[x[0], x[1]] == dkp.iloc[x[0], x[1]]]
graph_dkp = Graph()

for p in tqdm(pl):
    iterator = iter([0,1,0,-1,1,0,-1,0])
    i = 0
    while i < 4:
        i += 1
        n = (p[0]+next(iterator), p[1] + next(iterator))
        if n in pl:    
            graph_dkp.add_edge(p, n, 1)
graph_dkp

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]


{(1, 0): {(1, 1): 1}, (0, 1): {(0, 2): 1, (1, 1): 1}, (1, 1): {(1, 2): 1, (1, 0): 1, (0, 1): 1}, (0, 2): {(0, 1): 1, (1, 2): 1}, (1, 2): {(1, 1): 1, (0, 2): 1}}

In [13]:
def numcode(code_ : 'str') -> 'list':
    l_ = [(3, 2)]
    for x in code_:
        l_.append((int(np.where(nkp == x)[0]),int(np.where(nkp == x)[1])))
    return l_

# exemple :
t = '0299A'
numcode(t)

[(3, 2), (3, 1), (2, 1), (0, 2), (0, 2), (3, 2)]

In [14]:
def dircode(code_: 'list') -> 'list':
    l = [(0, 2)]
    for x in code_:
        l.append((int(np.where(dkp == x)[0]),int(np.where(dkp == x)[1])))
    return l

# exemple :
t = list('A>^^A')
dircode(t)

[(0, 2), (0, 2), (1, 2), (0, 1), (0, 1), (0, 2)]

In [9]:
def numpath(numcode: 'list') -> 'list':
    """
    dédouble les nodes
    """
    path = []
    #path.append([(3, 2)])
    i = 0
    iterator = iter(numcode)
    n1 = next(iterator)
    while i < len(numcode) - 1:
        n2 = next(iterator)
        p = find_path(graph_nkp, n1, n2).nodes
        if len(p) > 1:        
            path.append(find_path(graph_nkp, n1, n2).nodes)
        elif len(p) == 1:
            path.append(find_path(graph_nkp, n1, n2).nodes)
            path.append(find_path(graph_nkp, n1, n2).nodes)
        n1 = n2
        i += 1
    e = path[-1][-1]
    return list(chain.from_iterable(path)) + [e]
# exemple
#t = '0299A'
#numcode(t) = [(3, 2), (3, 1), (2, 1), (0, 2), (0, 2), (3, 2)]
numcode = [(3, 2), (3, 1), (2, 1), (0, 2), (0, 2), (3, 2)]
numpath(numcode)

[(3, 2),
 (3, 1),
 (3, 1),
 (2, 1),
 (2, 1),
 (2, 2),
 (1, 2),
 (0, 2),
 (0, 2),
 (0, 2),
 (0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (3, 2)]

In [10]:
def dirpath(dircode: 'list') -> 'list':
    """
    dédouble les nodes
    """
    path = []
    i = 0
    iterator = iter(dircode)
    n1 = next(iterator)
    while i < len(dircode) - 1:
        n2 = next(iterator)
        p = find_path(graph_dkp, n1, n2).nodes
        if len(p) > 1:        
            path.append(find_path(graph_dkp, n1, n2).nodes)
        elif len(p) == 1:
            path.append(find_path(graph_dkp, n1, n2).nodes)
            path.append(find_path(graph_dkp, n1, n2).nodes)
        n1 = n2
        i += 1
    e = path[-1][-1]
    return list(chain.from_iterable(path)) + [e]
    
# exemple
t = list('A>^^A')
dircode = [(0, 2), (0, 2), (1, 2), (0, 1), (0, 1), (0, 2)]
dirpath(dircode)

[(0, 2),
 (0, 2),
 (0, 2),
 (1, 2),
 (1, 2),
 (1, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 2),
 (0, 2)]

In [11]:
def sub_tup(a,b):
    return (a[0] - b[0], a[1] - b[1])

# exemple:
a = (3,1)
b = (1,0)
sub_tup(a,b)

(2, 1)

In [15]:
def rec(p : 'list') -> 'list':
        iterator = iter(p)
        d = {(0,1) : '>', (1,0) : 'v', (-1,0) : '^', (0,-1) : '<', (0,0) : 'A'}
        l = []
        c = 0
        next(iterator)
        for i in p[0:-1]:
            s = d[sub_tup(next(iterator),i)]
            # si la différence suivante est égale mettre un 'X'
            
            if s == 'A' and  l[-1] == 'A' and c == 0:
                c = 1
                #print('c=1')
                continue                
            elif s == 'A' and  l[-1] == 'A' and c == 1:
                l.append(s)
                c = 0
                continue
            
            l.append(s)
        return l
        
# exemple
p = numpath(numcode('029A'))
rec(p)    

['<', 'A', '^', 'A', '>', '^', '^', 'A', 'v', 'v', 'v', 'A']

In [16]:
liste = ['<', '^', '^','<','A', '^', 'A', 'v', '>',  'v', 'A', 'v', '>', 'v', 'A']
# 0 to 1
def range_01(liste_):
    liste = liste_
    for i in range(len(liste)):
        for j in range(5):
            if all([(x in ['<', '^','v']) for x in liste[i : i+j]]):
                liste[i : i+j] = sorted(liste[i : i+j])[::-1]
            elif all([(x in ['v', '>', '^']) for x in liste[i : i+j]]):
                liste[i : i+j] = sorted(liste[i : i+j])
    return liste

#exemple
liste = ['<', '^', '<', '^','A', 'v', '>', 'v', '>']
print('range_01 : ', range_01(liste))

def range_123(liste_):
    liste = liste_
    for i in range(len(liste_)):
        for j in range(1, 6, 1):
            if all([(x in ['<', 'v']) for x in liste [i : i+j]]):
                liste[i : i+j] = sorted(liste[i : i+j])[::-1]
                                
            elif all([(x in ['^', '>']) for x in liste [i : i+j]]):
                liste[i : i+j] = sorted(liste[i : i+j])
    return liste
        
#exemple
liste = ['<', 'v', '<', 'v','A', '^', '>', '^', '>']
print('range_123 : ', range_123(liste))

range_01 :  ['^', '^', '<', '<', 'A', '>', '>', 'v', 'v']
range_123 :  ['v', 'v', '<', '<', 'A', '>', '>', '^', '^']


In [17]:
# code
'029A'
# first
'<A^A>^^AvvvA' 
# second
'v<<A>>^A<A>AvA<^AA>A<vAAA>^A'
# third
'<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A'

'<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A'

In [18]:
s = 0
for c in input:
    print('code : ', c)
    p = numpath(numcode(c))
    i = rec(p)
    i = range_01(i)
    print(i)
    print('first : ', ''.join(i))
    i = dirpath(dircode(i))
    i = rec(i)
    i = range_123(i)
    print('second : ', ''.join(i))
    i = dirpath(dircode(i))
    i = rec(i)
    i = range_123(i)
    print('third : ', ''.join(i))
    s += len(i)*int(re.findall(r'\d+', c)[0])
    print(len(i), int(re.findall(r'\d+', c)[0]))
s

code :  029A
['<', 'A', '^', 'A', '>', '^', '^', 'A', 'v', 'v', 'v', 'A']
first :  <A^A>^^AvvvA
second :  v<<A>>^A<A>AvA<^AA>Av<AAA>^A
third :  v<A<AA>>^AvAA<^A>Av<<A>>^AvA^Av<A>^Av<<A>^A>AAvA^Av<A<A>>^AAAvA<^A>A
68 29
code :  980A
['^', '^', '^', 'A', '<', 'A', 'v', 'v', 'v', 'A', '>', 'A']
first :  ^^^A<AvvvA>A
second :  <AAA>Av<<A>>^Av<AAA>^AvA^A
third :  v<<A>>^AAAvA^Av<A<AA>>^AvAA<^A>Av<A<A>>^AAAvA<^A>Av<A>^A<A>A
60 980
code :  179A
['^', '<', '<', 'A', '^', '^', 'A', '>', '>', 'A', 'v', 'v', 'v', 'A']
first :  ^<<A^^A>>AvvvA
second :  <Av<AA>>^A<AA>AvAA^Av<AAA>^A
third :  v<<A>>^Av<A<A>>^AAvAA<^A>Av<<A>>^AAvA^Av<A>^AA<A>Av<A<A>>^AAAvA<^A>A
68 179
code :  456A
['^', '^', '<', '<', 'A', '>', 'A', '>', 'A', 'v', 'v', 'A']
first :  ^^<<A>A>AvvA
second :  <AAv<AA>>^AvA^AvA^Av<AA>^A
third :  v<<A>>^AAv<A<A>>^AAvAA<^A>Av<A>^A<A>Av<A>^A<A>Av<A<A>>^AAvA<^A>A
64 456
code :  379A
['^', 'A', '^', '^', '<', '<', 'A', '>', '>', 'A', 'v', 'v', 'v', 'A']
first :  ^A^^<<A>>AvvvA
second :  <A>A<AA

127900

In [ ]:
# quelquechose cloche avec le dernier c'est 64 et pas 68 normalement

In [19]:
from functools import cache

inputval = ""

posi = [
    ["7", "8", "9"],
    ["4", "5", "6"],
    ["1", "2", "3"],
    [None, "0", "A"],
]

arr_pads = [
    [None, "^", "A"],
    ["<", "v", ">"]
]

def get_pos(arr, code):
    for i, row in enumerate(arr):
        if code in row:
            return (i, row.index(code))

@cache
def shortest(start, end, layers):
    if start == "<" and end == ">":
        pass
    if isinstance(start, str):
        start = get_pos(arr_pads, start)
    if isinstance(end, str):
        end = get_pos(arr_pads, end)

    if layers == 0:
        return 1
    elif layers < 3:
        vert = None
        hori = None
        if end[0] < start[0]:
            vert = "^"
        elif end[0] > start[0]:
            vert = "v"
        if end[1] < start[1]:
            hori = "<"
        elif end[1] > start[1]:
            hori = ">"
        if not hori and not vert:
            return shortest("A", "A", layers - 1)
        elif not hori:
            return shortest("A", vert, layers - 1) + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + shortest(vert, "A", layers - 1)
        elif not vert:
            return shortest("A", hori, layers - 1) + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + shortest(hori, "A", layers - 1)
        else:
            if start[1] == 0:
                return shortest("A", hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, "A", layers - 1)
            elif end[1] == 0:
                return shortest("A", vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, "A", layers - 1)
            else:
                return min(
                    shortest("A", hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, "A", layers - 1),
                    shortest("A", vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, "A", layers - 1)
                )
    else:
        vert = None
        hori = None
        if end[0] < start[0]:
            vert = "^"
        elif end[0] > start[0]:
            vert = "v"
        if end[1] < start[1]:
            hori = "<"
        elif end[1] > start[1]:
            hori = ">"
        if not hori and not vert:
            return shortest("A", "A", layers - 1)
        elif not hori:
            return shortest("A", vert, layers - 1) + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + shortest(vert, "A", layers - 1)
        elif not vert:
            return shortest("A", hori, layers - 1) + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + shortest(hori, "A", layers - 1)
        else:
            if start[1] == 0 and end[0] == 3:
                return shortest("A", hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, "A", layers - 1)
            elif end[1] == 0 and start[0] == 3:
                return shortest("A", vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, "A", layers - 1)
            else:
                return min(
                    shortest("A", hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, "A", layers - 1),
                    shortest("A", vert, layers - 1) + \
                    (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1) + \
                    shortest(vert, hori, layers - 1) + \
                    (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1) + \
                    shortest(hori, "A", layers - 1)
                )

for start in ["<", "^", ">", "v", "A"]:
    for end in ["<", "^", ">", "v", "A"]:
        print(start, end, shortest(start, end, 1))

ut = input  # Replace input with predefined list

score = 0
for inputval in ut:
    intval = int(inputval[:3])
    total = 0
    for startp, endp in zip("A" + inputval[:3], inputval):
        total += shortest(get_pos(posi, startp), get_pos(posi, endp), 3)
    print(intval, total)
    score += intval * total
print(score)

< < 1
< ^ 3
< > 3
< v 2
< A 4
^ < 3
^ ^ 1
^ > 3
^ v 2
^ A 2
> < 3
> ^ 3
> > 1
> v 2
> A 2
v < 2
v ^ 2
v > 2
v v 1
v A 3
A < 4
A ^ 2
A > 2
A v 3
A A 1
29 68
980 60
179 68
456 64
379 64
126384
